In [ ]:
import utils.data_prc as dp 
import utils.build_network as bn
import utils.compilation_opt as cpo
from keras.models import load_model
%pylab inline

In [ ]:
Xtrain_pain, Xtest_pain, Xval_pain = dp.dataset_pickle_pain("pain_ds")
dp.dataset_pickle_emotions("fer2013", Xtrain_pain, Xtest_pain, Xval_pain)
dp.dataset_pickle_gsr('All_features')
dp.dataset_pickle_gsr_crossVal('GSR_ds')

In [ ]:
X_train_E, y_train_E, X_test_E, y_test_E, X_val_E, y_val_E = dp.dataset_loading("fer2013")
X_train, y_train, X_test, y_test, X_val, y_val = dp.dataset_loading("pain_ds")
X_train_g, y_train_g, X_test_g, y_test_g, X_val_g, y_val_g = dp.dataset_loading('All_features_noCrossVal')
X_train_gs, y_train_gs, X_test_gs, y_test_gs = dp.load_gsr_crossVal('GSR_ds')

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)
print(X_train_E.shape, y_train_E.shape, X_test_E.shape, y_test_E.shape, X_val_E.shape, y_val_E.shape)
print X_train_g.shape, y_train_g.shape, X_test_g.shape, y_test_g.shape,  X_val_g.shape, y_val_g.shape
print X_train_gs.shape, y_train_gs.shape, X_test_gs.shape, y_test_gs.shape

In [ ]:
emotion = { 0:'Angry', 1:'Disgust', 2: 'Fear', 3: 'Happy',
           4: 'Sad', 5: 'Surprise', 6: 'Neutral', 7: 'Pain'}

In [ ]:
num_classes = len(emotion)
samples_per_class = 7

for y, cls in emotion.items():
    idxs = np.flatnonzero(y_train_E == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=True)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(reshape(X_train_E[idx],(48,48)))
        plt.axis('off')
        if i == 0:
            plt.title(cls)

plt.show()


In [ ]:
NUM_CLASSES_EMOTION = 8
NUM_CLASSES_PAIN = 13
NUM_CLASSES_GSR = 5

y_train_E, y_test_E, y_val_E= dp.y_to_categorical(y_train_E, y_test_E, NUM_CLASSES_EMOTION, y_val_E)
y_train, y_test, y_val = dp.y_to_categorical(y_train, y_test, NUM_CLASSES_PAIN, y_val)
y_train_g, y_test_g, y_val_g = dp.y_to_categorical(y_train_g, y_test_g, NUM_CLASSES_GSR, y_val_g)
y_train_gs, y_test_gs = dp.y_to_categorical(y_train_gs, y_test_gs, NUM_CLASSES_GSR)

In [ ]:
#opt = cpo.sgdOpt()
#opt = cpo.rmsPropOpt()
#opt = cpo.adagradOpt() 
opt = cpo.adamOpt() 

#model = bn.inception_v3()
#model_1 = bn.build_pain_model(X_train)
#model = bn.build_model(X_train_E)
model_1 = bn.build_gsr_model()

model_1.summary()
model_1 = cpo.compiling(model_1, opt)

#model = cpo.compiling(model, opt)


In [ ]:
#model = load_model('model_emo_InceptionV3_nogcloud_30ep.h5')
#model_1 = load_model('model_pain_nogcloud_rmsprop_120ep.h5')
epochs = 120
batch_size = 32

print (X_train_gs.shape, y_train_gs.shape)

model_1 = cpo.training_cross_valid(model_1, batch_size, epochs, X_train_gs, y_train_gs)
#print (X_train_E.shape, y_train_E.shape)
#model, history = cpo.training(model, batch_size, epochs, X_train_E, y_train_E, X_val_E, y_val_E)
#model_1, history = cpo.training(model_1, batch_size, epochs, X_train_g, y_train_g, X_val_g, y_val_g)


In [ ]:
#cpo.eval_plot(model, X_val_E, y_val_E, history, epochs)
#cpo.eval_plot(model_1, X_val, y_val, history, epochs)
#print(model.predict_classes(X_train, batch_size=128))
#print(model.predict_classes(X_test, batch_size=128))

In [ ]:
model_1.save('model_finalSR_3densemodel_CrossVal.h5')  # creates a HDF5 file '.h5'
#del model  # deletes the existing model


In [ ]:
scores = model_1.evaluate(X_test_gs, y_test_gs, batch_size = 32,verbose=0)
print scores